# Первая БД

In [1]:
import sqlite3 as sl

In [2]:
authors = sl.connect("authors_database.db")

In [3]:
cursor = authors.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

In [4]:
cursor.execute('''
CREATE TABLE author (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    login TEXT NOT NULL,
    email TEXT NOT NULL
);
''')

In [5]:
cursor.execute('''
CREATE TABLE blog (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    owner_id INTEGER NOT NULL,
    name TEXT NOT NULL,
    description TEXT,
    FOREIGN KEY (owner_id) REFERENCES author(id)
);
''')

In [6]:
cursor.execute('''
CREATE TABLE post (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    header TEXT NOT NULL,
    text TEXT NOT NULL,
    author_id INTEGER NOT NULL,
    blog_id INTEGER NOT NULL,
    FOREIGN KEY (author_id) REFERENCES author(id),
    FOREIGN KEY (blog_id) REFERENCES blog(id)
);
''')

In [7]:
# По заданию нужны сообщения, пришлось собрать еще одну табличку, связанную с постом
cursor.execute('''
CREATE TABLE comment (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text TEXT NOT NULL,
    author_id INTEGER NOT NULL,
    post_id INTEGER NOT NULL,
    FOREIGN KEY (author_id) REFERENCES author(id) ON DELETE CASCADE,
    FOREIGN KEY (post_id) REFERENCES post(id) ON DELETE CASCADE
);
''')

In [8]:
authors.commit()
authors.close()

# Делаем вторую бд 

In [9]:
logs = sl.connect("logs_database.db")

In [10]:
cursor = logs.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

In [11]:
cursor.execute('''
CREATE TABLE space_type (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);
''')

In [12]:
cursor.execute('''
INSERT INTO space_type (name) VALUES ('global'), ('blog'), ('post');
''')

In [13]:
cursor.execute('''
CREATE TABLE event_type (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);
''')

In [14]:
cursor.execute('''
INSERT INTO event_type (name) VALUES ('login'), ('comment'), ('create_post'), ('delete_post'), ('logout');
''')

In [15]:
cursor.execute('''
CREATE TABLE logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    datetime DATETIME NOT NULL,
    user_id INTEGER NOT NULL,
    space_type_id INTEGER NOT NULL,
    event_type_id INTEGER NOT NULL,
    FOREIGN KEY (space_type_id) REFERENCES space_type(id),
    FOREIGN KEY (event_type_id) REFERENCES event_type(id)
);
''')

In [16]:
logs.commit()
logs.close()

# Заполняем базы тестовыми значениями

In [17]:
from datetime import datetime as dt
import random

authors = sl.connect("authors_database.db")
auth = authors.cursor()

logs = sl.connect("logs_database.db")
log = logs.cursor()

# login
# comment
# create_post
# delete_post
# logout

class logger:
    @staticmethod
    def createPost(login: str):
        auth.execute('''
        SELECT * FROM author WHERE login = ?
        ''', (login, ))
        id = auth.fetchall()[0][0]
        log.execute('''
        INSERT INTO logs (datetime, user_id, space_type_id, event_type_id) VALUES (
            ?,
            ?,
            (SELECT id FROM space_type WHERE name = 'blog'),
            (SELECT id FROM event_type WHERE name = 'create_post')
        );
        ''', (getDatetimeInRightWay(), id))
        logs.commit()

    @staticmethod
    def deletePost(login: str):
        auth.execute('''
        SELECT * FROM author WHERE login = ?
        ''', (login, ))
        id = auth.fetchall()[0][0]
        log.execute('''
        INSERT INTO logs (datetime, user_id, space_type_id, event_type_id) VALUES (
            ?,
            ?,
            (SELECT id FROM space_type WHERE name = 'blog'),
            (SELECT id FROM event_type WHERE name = 'delete_post')
        );
        ''', (getDatetimeInRightWay(), id))
        logs.commit()

    @staticmethod
    def login(_login: str):
        auth.execute('''
        SELECT * FROM author WHERE login = ?
        ''', (_login, ))
        id = auth.fetchall()[0][0]
        log.execute('''
        INSERT INTO logs (datetime, user_id, space_type_id, event_type_id) VALUES (
            ?,
            ?,
            (SELECT id FROM space_type WHERE name = 'global'),
            (SELECT id FROM event_type WHERE name = 'login')
        );
        ''', (getDatetimeInRightWay(), id))
        logs.commit()

    @staticmethod
    def logout(_login: str):
        auth.execute('''
        SELECT * FROM author WHERE login = ?
        ''', (_login, ))
        id = auth.fetchall()[0][0]
        log.execute('''
        INSERT INTO logs (datetime, user_id, space_type_id, event_type_id) VALUES (
            ?,
            ?,
            (SELECT id FROM space_type WHERE name = 'global'),
            (SELECT id FROM event_type WHERE name = 'logout')
        );
        ''', (getDatetimeInRightWay(), id))
        logs.commit()

    @staticmethod
    def comment(login: str):
        auth.execute('''
        SELECT * FROM author WHERE login = ?
        ''', (login, ))
        id = auth.fetchall()[0][0]
        log.execute('''
        INSERT INTO logs (datetime, user_id, space_type_id, event_type_id) VALUES (
            ?,
            ?,
            (SELECT id FROM space_type WHERE name = 'post'),
            (SELECT id FROM event_type WHERE name = 'comment')
        );
        ''', (getDatetimeInRightWay(), id))
        logs.commit()

def getDatetimeInRightWay():
    return dt.now().strftime("%Y-%m-%d %H:%M:%S")

def isLoginInBase(login: str):
    auth.execute(f'''
    SELECT * FROM author WHERE login = ?
    ''', (login,))
    return auth.fetchall()

def setLoginInBase(login: str, email: str = ""):
    if not email: email = f"{login}@mail.ru"
    auth.execute('''
    INSERT INTO author (login, email) VALUES (?, ?);
    ''', (login, email))
    return authors.commit()


def createBlog(login: str, name: str, description: str):
    if not isLoginInBase(login):
        setLoginInBase(login)
    auth.execute('''
    INSERT INTO blog (owner_id, name, description) VALUES (
        (SELECT id from author where login = ?),
        ?,
        ?
    );
    ''', (login, name, description))
    
    return authors.commit()


# Вот тут я немного не понял, вроде как были только посты, блог и пользователи, откуда взялись комменты - вопрос интересный.
def createPost(login: str, header: str, text: str, blog_name: int):
    if not isLoginInBase(login):
        setLoginInBase(login)
    
    logger.createPost(login) #logger
    
    auth.execute('''
    INSERT INTO post (header, text, author_id, blog_id) VALUES (
        ?,
        ?,
        (SELECT id from author where login = ?),
        (SELECT id from blog where name = ?)
    );
    ''', (header, text, login, blog_name))
    return authors.commit()


# Да-да, хедеры могут быть не уникальные, так просто проще реализовать простотип
def deletePost(login: str, header: str):
    if not isLoginInBase(login):
        setLoginInBase(login)
    
    logger.deletePost(login) #logger
    
    auth.execute('''
    DELETE FROM post WHERE id = (SELECT id FROM post WHERE header = ?);
    ''', (header,))
    return authors.commit()

def createComment(login: str, text: str, post_header: str):
    if not isLoginInBase(login):
        setLoginInBase(login)

    logger.comment(login) #logger

    auth.execute('''
    INSERT INTO comment (text, author_id, post_id) VALUES (
        ?,
        (SELECT id from author where login = ?),
        (SELECT id from post WHERE header = ?)
    );
    ''', (text, login, post_header))
    return authors.commit()

def login_foo(login: str):
    if not isLoginInBase(login):
        setLoginInBase(login)

    logger.login(login) #logger

def logout(login: str):
    if not isLoginInBase(login):
        setLoginInBase(login)

    logger.logout(login) #logger


for i in range(100):
    login = format(random.getrandbits(64), "x")
    login_foo(login)
    if random.choice([0, 1]):
        createBlog(login, login + "_name", "All about " + login)
        createPost(login, login + "_post", login + " thinks that...", login + "_name")
        if random.choice([0, 1]):
            deletePost(login, login + "_post")
        else:
            createComment(login, login + "_text", login + "_post")
    logout(login)

logs.close()
authors.close()